In [1]:
# import argparse, sys

# parser=argparse.ArgumentParser()

# parser.add_argument("-b", "--base_path", help="S3 bucket base path", required=True)
# parser.add_argument("-s", "--start", help="Start date to calculate", required=True)
# parser.add_argument("-e", "--end", help="End date to calculate", required=True)

# args=parser.parse_args()

# base_path = args.base_path
# start = args.start
# end = args.end

In [2]:
base_path = "."
start = "2020-10-31"
end = "2020-10-31"

In [3]:
from datetime import datetime, timedelta

def get_dates_in_range(start, end):
    # Convert start and end strings to datetime objects
    start_date = datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.strptime(end, "%Y-%m-%d")

    # Create a list to store the dates within the range
    dates_list = []

    # Loop through the dates and append them to the list
    current_date = start_date
    while current_date <= end_date:
        dates_list.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)

    return dates_list

In [4]:
dates = get_dates_in_range(start, end)

num_of_date = len(dates)
time_range = ""

if(num_of_date == 1):
    time_range = "date"
elif(num_of_date < 7 or num_of_date == 7):
    time_range = "week"
else:
    time_range = "month"

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DecimalType, DoubleType
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession \
    .builder \
    .appName("UAW_Calculation") \
    .getOrCreate()

23/07/17 10:21:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
pre_tx_df = spark.read.format("csv") \
    .option("header", True) \
    .load(list(map(lambda date: "{base_path}/pre-tx/date={date}/*.csv".format(base_path = base_path, date = date), dates)))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [8]:
contract_df = spark.read.format("parquet") \
    .load("{base_path}/contracts/*/*.parquet".format(base_path = base_path))

In [9]:
from pyspark.sql.functions import lit, col
import time

# add "True col to contract df"
contract_df = contract_df.withColumnRenamed('address', 'from_address') \
    .withColumn("check", lit(True)) 

In [10]:
start_time = time.time()

result_df = pre_tx_df \
    .sort('to_address') \
    .select('from_address', 'to_address') \
    .filter(col("from_address") != '0x0000000000000000000000000000000000000000') \
    .filter(col("from_address") != '0x000000000000000000000000000000000000dead') \
    .filter(col("to_address") != '0x0000000000000000000000000000000000000000') \
    .filter(col("to_address") != '0x000000000000000000000000000000000000dead') \
    .distinct() \
    .join(contract_df,on='from_address',how='left') \
    .filter("check is null") \
    .sort("to_address") \
    .groupBy("to_address") \
    .count() \
    .withColumnRenamed('to_address', 'Address') \
    .withColumnRenamed('count', 'UAW') 

time.time() - start_time

0.16849946975708008

In [11]:
# result_df.show(10, False)

In [12]:
# start_time = time.time()

# uw_result_df = pre_tx_df \
#     .sort('to_address') \
#     .select('from_address', 'to_address') \
#     .filter(col("from_address") != '0x0000000000000000000000000000000000000000') \
#     .filter(col("from_address") != '0x000000000000000000000000000000000000dead') \
#     .filter(col("to_address") != '0x0000000000000000000000000000000000000000') \
#     .filter(col("to_address") != '0x000000000000000000000000000000000000dead') \
#     .distinct() \
#     .join(contract_df,on='from_address',how='left') \
#     .filter("check is null") \
#     .sort("to_address") \
#     .groupBy("to_address") \
#     .count() \
#     .withColumnRenamed('to_address', 'Address') \
#     .withColumnRenamed('count', 'UAW') 

# time.time() - start_time

0.06892871856689453

In [13]:
# start_time = time.time()

# from_addr_df = pre_tx_df \
#     .select('from_address') \
#     .distinct() \
#     .withColumnRenamed('from_address', 'Address')

# to_addr_df = pre_tx_df \
#     .select('to_address') \
#     .distinct() \
#     .withColumnRenamed('to_address', 'Address')

# contract_addr_df = pre_tx_df \
#     .select('token_contract') \
#     .distinct() \
#     .withColumnRenamed('token_contract', 'Address')

# uw_df = from_addr_df \
#     .union(to_addr_df) \
#     .union(contract_addr_df) \
#     .distinct()

# start_time = time.time()

In [14]:
# uw_df.count()

In [15]:
start_time = time.time()

result_df.repartition(1) \
    .write \
    .option("header",True) \
    .csv(
        "{base_path}/uaw/{time_range}/start={start}_end={end}/" \
            .format(
                base_path = base_path, \
                time_range = time_range, \
                start = start, \
                end = end \
            ) \
    )

time.time() - start_time

6.329843044281006

In [16]:
spark.stop()